# LSTM

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
cd '/content/drive/MyDrive/Class/Energy Technology and Management/Topic 05 - Project'

In [ ]:
!ls

# Load data

In [ ]:
import numpy as np
import pandas as pd
df_avg = pd.read_csv('data_processed.csv', parse_dates=['datetime'], index_col='datetime')
df_avg.head()

## \[Mod\] Change the frequency of datetime here.
- https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

In [ ]:
df_avg.index.freq = 'D'

In [ ]:
import matplotlib.pyplot as plt
df_avg.plot(figsize=(10, 3))
plt.show()

# Scaling data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_avg['y (scaled)'] = scaler.fit_transform(df_avg[['y']])
display(df_avg.head())

# Converting time series into supervised data

## \[Mod\] Change the window size here.

In [ ]:
window_size = 10

In [ ]:
dft = df_avg.copy()
dft['split'] = ''
for i in range(1, window_size + 1):
    col = f"t-{i}" 
    dft[col] = dft.iloc[:,1].shift(i)
dft = dft.dropna()

# Split data into training and testing set

In [ ]:
test_size = 0.2
n_rows = dft.shape[0]
train_size = int(n_rows * (1-test_size))
test_size = n_rows - train_size
dft.iloc[0:train_size, 2] = 'train'
dft.iloc[train_size:, 2] = 'test'

In [ ]:
filt = dft['split'] == 'train'
X_train = dft[filt].iloc[:,3:].values
y_train = dft[filt].iloc[:,1].values
#
filt = dft['split'] == 'test'
X_test = dft[filt].iloc[:,3:].values
y_test = dft[filt].iloc[:,1].values
#
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# Reshape input data for LSTM traning
- Dimension: \[samples, time steps, features\]


In [ ]:
X_train_rs = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_rs = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
#
print(X_train_rs.shape)
print(X_test_rs.shape)

# Training model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.backend import clear_session

## \[Mod\] Change the model architecture here

In [ ]:
# Clear session
clear_session()

# Change the network architecture here
model = Sequential()
model.add(LSTM(200, input_shape=(X_train_rs.shape[1], X_train_rs.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))

In [ ]:
print(model.summary())

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')

# Training

In [ ]:
history = model.fit(X_train_rs, y_train, epochs=100, batch_size=10, validation_data=(X_test_rs, y_test), 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=20)], verbose=1, shuffle=False)

model.summary()

# Predictions

In [ ]:
# Creating new DataFrame to store results
dft_result = dft.iloc[:,0:3]

# Model predictions
y_pred_train = model.predict(X_train_rs)
y_pred_test = model.predict(X_test_rs)
y_pred = np.concatenate((y_pred_train, y_pred_test), axis=0)

# Invert predictions
y_pred_train_pl = scaler.inverse_transform(y_pred_train)
y_pred_test_pl = scaler.inverse_transform(y_pred_test)
y_pred_pl = scaler.inverse_transform(y_pred)

# Storing results
dft_result['y_pred (scaled)'] = y_pred 
dft_result['y_pred'] = y_pred_pl

# Evaluation

In [ ]:
def model_eval(df_eval):
    MAE = df_eval['error'].abs().mean()
    RMSE = np.sqrt((df_eval['error']**2).mean())
    MAPE = df_eval['percentage'].abs().mean()
    print(f"-------")
    print(f"Mean absolute error: {MAE:6.3f}")
    print(f"Root mean squared error: {RMSE:6.3f}")
    print(f"Mean absolute percentage error: {MAPE:6.3f}")

In [ ]:
dft_result['error'] = dft_result['y'] - dft_result['y_pred']
dft_result['percentage'] = dft_result['error']/dft_result['y']*100

dft_eval = dft_result[dft_result['split'] == 'train']
print(f"Training")
model_eval(dft_eval)

print(f"\nTesting")
dft_eval = dft_result[dft_result['split'] == 'test']
model_eval(dft_eval)

print(f"\nAll")
model_eval(dft_result)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,5))

dft_plot = dft_result[dft_result['split'] == 'train']
dft_plot[['y', 'y_pred']].plot(ax=ax1)
ax1.set_title('Training data')

dft_plot = dft_result[dft_result['split'] == 'test']
dft_plot[['y', 'y_pred']].plot(ax=ax2)
ax2.set_title('Testing data')

plt.show()

# Forecasting

## Retrain the model with entire data

In [ ]:
# Extract data
X_train = dft.iloc[:,3:].values
y_train = dft.iloc[:,1].values

# Reshape data
X_train_rs = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])

print(X_train_rs.shape)
print(y_train.shape)

In [ ]:
history = model.fit(X_train_rs, y_train, epochs=100, batch_size=10, validation_data=(X_test_rs, y_test), 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=20)], verbose=1, shuffle=False)

model.summary()

## Storing prediction result

In [ ]:
y_pred = model.predict(X_train_rs)
y_pred_pl = scaler.inverse_transform(y_pred)
dft_result['y_pred_2 (scale)'] = y_pred
dft_result['y_pred_2'] = y_pred_pl
display(dft_result.head())

## Forecasting

In [ ]:
n_forecast = 10

In [ ]:
y = dft_result['y (scaled)'].values
freq = dft_result.index.freq
dt_last = dft_result.index[-1]
dts = []
for i in range(1,n_forecast+1):
    x = y[-window_size-1:-1]
    x = x.reshape(1,1,-1)
    y = np.append(y,model.predict(x).flatten())
    dts.append(dt_last + freq * i)

y_fore = y[-n_forecast-1:-1]
y_fore_pl = scaler.inverse_transform([y_fore]).ravel()

dft_forecast = pd.DataFrame( {'datetime': dts, 'y_pred (scaled)': y_fore, 'y_pred': y_fore_pl} )
dft_forecast = dft_forecast.set_index('datetime')
dft_forecast.head()

# Plotting

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
dft_result['y'].plot(ax=ax)
dft_result['y_pred_2'].plot(ax=ax)
dft_forecast['y_pred'].plot(ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
dft_result['y'].plot(ax=ax)
dft_result['y_pred_2'].plot(ax=ax)
dft_forecast['y_pred'].plot(ax=ax,linestyle='--',marker='s')

dt_start = dft_result.index[-n_forecast*8]
dt_end = dft_forecast.index[-1]
ax.set_xlim(dt_start,dt_end)

# Writing data to files

In [ ]:
dft_result.to_csv('data_LSTM_predict.csv')

In [ ]:
dft_forecast.to_csv('data_LSTM_forecast.csv')